In [1]:
text ="#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone"

In [2]:
from spacy.lang.en import English

In [3]:
nlp = English()

In [4]:
from spacy.lang.en.stop_words import STOP_WORDS

In [13]:
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [14]:
spacy_tokenizer(text)

['fingerprint',
 'pregnancy',
 'test',
 'https://goo.gl/h1mfqv',
 'android',
 'app',
 'beautiful',
 'cute',
 'health',
 'igers',
 'iphoneonly',
 'iphonesia',
 'iphone']

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [16]:
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [17]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

In [18]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [19]:
train_df = pd.read_csv('train.csv')

In [20]:
train_df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [22]:
from sklearn.model_selection import train_test_split

X = train_df['tweet'] # the features we want to analyze
ylabels = train_df['label'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [23]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x00000254F431C0C8>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x00000254E7D840D8>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_

In [24]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.882996632996633
Logistic Regression Precision: 0.7888086642599278
Logistic Regression Recall: 0.7307692307692307


In [25]:
from sklearn.naive_bayes import MultinomialNB

In [41]:
clf = MultinomialNB()

In [42]:
pipe_mnb = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('clf', clf)])

# model generation
pipe_mnb.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x00000254F7B64288>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x00000254E7D840D8>,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [43]:
predicted = pipe_mnb.predict(X_test)

# Model Accuracy
print("Multinomial Naive Bayes Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Multinomial Naive Bayes Precision:",metrics.precision_score(y_test, predicted))
print("Multinomial Naive Bayes Recall:",metrics.recall_score(y_test, predicted))

Multinomial Naive Bayes Accuracy: 0.890993265993266
Multinomial Naive Bayes Precision: 0.7765089722675367
Multinomial Naive Bayes Recall: 0.7959866220735786


In [44]:
from sklearn.model_selection import GridSearchCV

In [45]:
parameters_mnb = {'clf__alpha': (0.1, 0.25, 0.5, 0.75)}

In [46]:
grid_search = GridSearchCV(pipe_mnb,
                           parameters_mnb,
                           n_jobs=-1,
                           verbose=1,
                           cv=5)

In [47]:
 grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('cleaner',
                                        <__main__.predictors object at 0x00000254F7B64288>),
                                       ('vectorizer',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                    

In [49]:
grid_search.best_estimator_

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x00000254F6E24788>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x00000254E7D840D8>,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=0.75, class_prior=None, fit_prior=True))],
         verbose=False)

In [50]:
grid_search.best_estimator_.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x00000254F6E24788>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x00000254E7D840D8>,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=0.75, class_prior=None, fit_prior=True))],
         verbose=False)

In [51]:
predicted = grid_search.best_estimator_.predict(X_test)

# Model Accuracy
print("Multinomial Naive Bayes Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Multinomial Naive Bayes Precision:",metrics.precision_score(y_test, predicted))
print("Multinomial Naive Bayes Recall:",metrics.recall_score(y_test, predicted))

Multinomial Naive Bayes Accuracy: 0.8922558922558923
Multinomial Naive Bayes Precision: 0.7598784194528876
Multinomial Naive Bayes Recall: 0.8361204013377926
